In [ ]:
import easyocr
import cv2
import time
import psutil
import concurrent.futures

# Function to measure resource utilization
def measure_resource_utilization():
    cpu_usage = psutil.cpu_percent()
    memory_usage = psutil.virtual_memory().percent
    return cpu_usage, memory_usage

# Function to process a batch of frames with OCR
def process_frame_batch(frame_batch, reader_cpu):
    results_batch = []
    for frame in frame_batch:

        result = reader_cpu.readtext(frame)
        results_batch.append(result)
    return results_batch

def process_video_cpu_optimized(video_path, output_path, batch_size=4, max_workers=2):
    reader_cpu = easyocr.Reader(['en'], gpu=False)
    cap = cv2.VideoCapture(video_path)
    outputs_length = 0
    start_time = time.time()

    # Create a VideoWriter to save the processed video
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

    frame_batch = []
    futures = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frameBW = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame_batch.append(frameBW)
            
            # Process batch when batch size is reached
            if len(frame_batch) == batch_size:
                futures.append(executor.submit(process_frame_batch, frame_batch.copy(), reader_cpu))
                frame_batch.clear()

        # Process any remaining frames
        if frame_batch:
            futures.append(executor.submit(process_frame_batch, frame_batch.copy(), reader_cpu))

        # Collect and process results
        for future in concurrent.futures.as_completed(futures):
            results_batch = future.result()
            for result in results_batch:
                outputs_length += len(result)

                for (bbox, text, prob) in result:
                    (top_left, top_right, bottom_right, bottom_left) = bbox
                    cv2.rectangle(frame, tuple(map(int, top_left)), tuple(map(int, bottom_right)), (0, 255, 0), 2)
                    cv2.putText(frame, text, (int(top_left[0]), int(top_left[1] - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                
                out.write(frame)

    cap.release()
    out.release()

    elapsed_time = time.time() - start_time
    fps = outputs_length / elapsed_time if elapsed_time > 0 else 0
    cpu_usage, memory_usage = measure_resource_utilization()

    print(f"Optimized CPU Results: Outputs Length: {outputs_length}, FPS: {fps:.2f}, CPU Usage: {cpu_usage}%, Memory Usage: {memory_usage}%")

# Specify video path and output
video_path = 'video.mp4'  # Replace with your video file
output_path_cpu = 'optimized_output_cpu.avi'

# Process the video using optimized batch size and limited threads
process_video_cpu_optimized(video_path, output_path_cpu, batch_size=4, max_workers=2)
